In [1]:
LANG = 'es' # or 'es'
TASK = 'gender' # or 'bot_human'

In [2]:
# Parameters
LANG = "en"
TASK = "gender"


In [3]:
import os
from fastai.text import * 

In [4]:
torch.cuda.device_count()

1

In [5]:
proj_path = Path(os.getcwd()).parents[1]

In [6]:
data_path = proj_path/f'clean_data_csv_{LANG}'

In [7]:
df = pd.read_csv(data_path/f'all_{TASK}_data.csv')

In [8]:
df.head()

,gender,text
0,male,"#NHLAllStar 2018 WHAT IF:East vs west, but the..."
1,female,"Blue Button 2.0 is a developer-friendly, stand..."
2,male,Planting churches isn’t about writing checks. ...
3,female,A bunch of kids just walked through the @POLIT...
4,female,@Charlottie_96 @Jennamill6 🙌🏼


In [9]:
data_lm = TextLMDataBunch.from_csv(data_path, f'all_{TASK}_data.csv')

In [10]:
data_clas = TextClasDataBunch.from_csv(data_path, f'all_{TASK}_data.csv', 
                                       vocab=data_lm.train_ds.vocab, bs=32)

In [11]:
data_lm.save(f'data_lm_export_{TASK}_{LANG}.pkl')
data_clas.save(f'data_clas_export_{TASK}_{LANG}.pkl')

In [12]:
data_lm = load_data(data_path, fname=f'data_lm_export_{TASK}_{LANG}.pkl')
data_clas = load_data(data_path, fname=f'data_clas_export_{TASK}_{LANG}.pkl', bs=16)

In [13]:
if LANG == 'es':
    weights_pretrained = 'model-30k-vocab-noqrnn'
    itos_pretrained = 'itos_pretrained'
    pretained_data = (weights_pretrained, itos_pretrained)
    learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5, pretrained_fnames=pretained_data)
else:
    learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
1,4.557302,4.200167,0.343832,02:29
2,4.273669,3.999150,0.356987,02:30
3,4.187714,3.977993,0.357642,02:30
4,4.127624,3.958965,0.359653,02:30
5,4.043993,3.939752,0.362090,02:32
6,3.966621,3.916475,0.363950,02:31
7,3.880331,3.894858,0.366267,02:30
8,3.788175,3.880169,0.367872,02:30
9,3.715978,3.874094,0.368322,02:30
10,3.679970,3.873401,0.368425,02:30


In [14]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
1,3.527667,3.809379,0.374650,02:47
2,3.484154,3.757525,0.380674,02:47
3,3.430391,3.728460,0.384667,02:47
4,3.382685,3.707597,0.388116,02:47
5,3.291720,3.705999,0.389598,02:47
6,3.215430,3.712539,0.390667,02:47
7,3.148415,3.714983,0.391520,02:47
8,3.065242,3.724349,0.391762,02:47
9,3.001264,3.732020,0.391900,02:47
10,2.985130,3.736702,0.391613,02:47


In [15]:
if LANG == 'es':
    test_string = "Nació en 1910"
else:
    test_string = "He was born in 1910"
learn.predict(test_string, n_words=50)

"He was born in 1910 and gave birth in 1948 . a voice there for her fam . Is she going to die do n't leave the earthquake in seven years ? https : / / t.co / https : / / t.co / micro - m - a - hole . 3D"

In [16]:
learn.save_encoder(f'ft_enc_{TASK}_{LANG}')

In [17]:
learn_class = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn_class.load_encoder(f'ft_enc_{TASK}_{LANG}')

In [18]:
data_clas.show_batch()

text,target
xxbos @pizster @bendleshnitz xxup proud ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤,male
"xxbos xxmaj ex - xxmaj xxunk xxup ceo : xxmaj as a xxup ceo and member of the xxmaj board of xxmaj directors at a public company , i can tell you that if we had an increase in profitability we would have been delighted but it would not lead in and of itself to more hiring or an increase in xxunk : / / t.co / xxunk",male
"xxbos xxmaj why are there so many adults tweeting about kids re : b - xxmaj ball ? xxmaj ya'll are xxmaj xxunk depressing . xxmaj kids do n't do this , kids today ai nt tough , kids transfer to much , kids only want to go xxup d1 . xxup smh , kids do not respond well to that constant negative preaching ( never have )",male
"xxbos xxmaj snapchat adds a 2017 year in review feature for saved memories - xxmaj snapchat users who snap and save can take a stroll down memory lane with the app ’s new feature , “ a xxmaj look xxmaj back at 2017 . ” xxmaj if you ’re the type that would rather xxunk than remain committe ... https : / / t.co / xxunk",female
xxbos xxmaj that was fun 😂 😂 please join me tomorrow for round two . i 'm going to attempt to stream as long as possible . xxmaj can i beat today ... xxmaj hmmm . # xxmaj witcher3 # thewitcher # twitch # girlgamer # canadaday goodnight everyone ! xxmaj until xxunk ( afternoon sometime ) i 'll try my best for noon .,female


In [19]:
learn_class.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
1,0.634418,0.593439,0.682144,02:49
2,0.646839,0.599347,0.675785,02:48
3,0.646946,0.636670,0.656513,02:45
4,0.641460,0.625001,0.673042,02:46
5,0.643585,0.661970,0.657921,02:46
6,0.630972,0.605278,0.677144,02:45
7,0.620150,0.583143,0.688527,02:48
8,0.629080,0.577959,0.687896,02:44
9,0.616440,0.574907,0.691585,02:44
10,0.616854,0.572913,0.693794,02:51


In [20]:
learn.save(f'model_stage_1_{TASK}_{LANG}')